In [ ]:
import sys
import os

sys.path.append('..')

import utilities.functions as functions

from utilities.functions import (
    load_data,
    check_key_uniqueness,
    save_parquet,
    merge_df,
    load_orders,
    process_orders_pandas,
)

Load the data --- se necessario salvar em stage - neste momento o estara comentado

In [ ]:
URL_CONSUMER = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
df_consumer = load_data(URL_CONSUMER)[["customer_id", "active","created_at"]]

In [ ]:
URL_RESTAURANT ="https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
df_restaurant= load_data(URL_RESTAURANT)

In [ ]:
ab_test_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
df_ab = load_data(ab_test_url)

In [ ]:
#save_parquet(df_consumer, "stage", "df_consumer.parquet")
#save_parquet(df_restaurant, "stage", "df_restaurant.parquet")
#save_parquet(df_consumer, "stage", "df_consumer.parquet")

Bronze layer - verify duplicates e nulo. e se necessario remover

In [ ]:
check_key_uniqueness(df_consumer, ["customer_id","active","created_at"])

In [ ]:
check_key_uniqueness(df_restaurant, ["id"])

In [ ]:
check_key_uniqueness(df_ab, ["customer_id","is_target"])
print(df_ab[df_ab["customer_id"].isna()])


In [ ]:
df_ab_np = df_ab[~df_ab["customer_id"].isna()]

Antes de carregar a base ordens sera definido o publico todal, e da base de ordem serao filtrados somentes os clientes elegiceis

In [ ]:
df_publico=merge_df(df_ab_np,df_consumer,['customer_id'],'outer')

In [ ]:
clientes_mes = (
    df_publico.groupby(['active', 'is_target'], dropna=False)
              ['customer_id']
              .nunique()
              .reset_index(name='numero_clientes_distintos')
)
clientes_mes

In [ ]:
df_publico = df_publico.dropna(subset=['active'])

In [ ]:
clientes_mes = (
    df_publico.groupby(['active', 'is_target'], dropna=False)
              ['customer_id']
              .nunique()
              .reset_index(name='numero_clientes_distintos')
)
clientes_mes

Publico definido e todos os clientes marcados no teste a/b e existentes na base de clientes

In [ ]:
save_parquet(df_publico, "silver", "df_publico.parquet")

Da base de ordens serao filtrados todos os clientes com orden nos meses de de dezembro e janeiro

In [ ]:
URL_ORDERS = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"

COLUMNS_TO_DROP = [
    'cpf','customer_name','delivery_address_city','delivery_address_country',
    'delivery_address_district','delivery_address_external_id',
    'delivery_address_latitude','delivery_address_longitude',
    'delivery_address_state','delivery_address_zip_code','items',
    'merchant_latitude','merchant_longitude','merchant_timezone',
    'order_scheduled','order_scheduled_date'
]

customer_ids = df_ab["customer_id"].astype(str).unique()

df_orders = load_orders(
    url=URL_ORDERS,
    customer_ids=customer_ids,
    columns_to_drop=COLUMNS_TO_DROP
)

df_orders.head()


In [ ]:
print(check_key_uniqueness(df_orders, ["customer_id","merchant_id","order_id"]))
print(check_key_uniqueness(df_orders, ["customer_id","merchant_id","order_id","order_created_at"]))

In [ ]:
#save_parquet(df_orders, "stage", "df_orders.parquet")

Add orders para a base de publico

In [ ]:
df_publico_orders=merge_df(df_publico,df_orders,['customer_id'],'inner')

In [ ]:
df_publico_orders.head()

Construcao de chave unica, e sumarizacoes visao cliente

In [ ]:
df_publico=process_orders_pandas(df_publico_orders)

In [ ]:
save_parquet(df_publico, "silver", "df_publico.parquet")

Uma linha por cliente, com as variaveis necessarias

In [ ]:
df_pub_un = df_publico[["customer_id",'origin_platform',"is_target", "order_created_month", "num_pedidos_mes", "num_pedidos_hist",'total_amount_mes','ticket_medio']].drop_duplicates().reset_index(drop=True)
df_pub_un.head()

Salvar base visao cliente em gold layer

In [ ]:
save_parquet(df_pub_un, "gold", "df_pub_un.parquet")